RNN 코드구현

In [ ]:
import numpy as np

def softmax(x):
   e_x = np.exp(x - np.max(x, axis=0, keepdims=True))
   return e_x / np.sum(e_x, axis=0, keepdims=True)


In [ ]:
def rnn_cell_forward(xt, a_prev, parameters):
   Wax = parameters["Wax"]
   Waa = parameters["Waa"]
   Wya = parameters["Wya"]
   ba = parameters["ba"]
   by = parameters["by"]
   
   a_next = np.tanh(Wax @ xt + Waa @ a_prev + ba)
   yt_pred = softmax(Wya @ a_next + by)
   
   cache = (a_next, a_prev, xt, parameters)
   return a_next, yt_pred, cache


In [ ]:
def rnn_forward(x, a0, parameters):

    caches = []
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    a_next = a0
    for t in range(T_x):
        xt = x[:, :, t]
        a_next, yt_pred, cache = rnn_cell_forward(xt, a_next, parameters)
        a[:, :, t] = a_next
        y_pred[:, :, t] = yt_pred
        caches.append(cache)
    
    caches = (caches, x)
    return a, y_pred, caches


In [ ]:
# 테스트 코드
np.random.seed(1)
n_x, m, T_x = 3, 10, 4
n_a, n_y = 5, 2
x_test = np.random.randn(n_x, m, T_x)
a0_test = np.random.randn(n_a, m)
parameters_test = {
    "Waa": np.random.randn(n_a, n_a),
    "Wax": np.random.randn(n_a, n_x),
    "Wya": np.random.randn(n_y, n_a),
    "ba": np.random.randn(n_a, 1),
    "by": np.random.randn(n_y, 1)
}
a_out, y_pred_out, caches_out = rnn_forward(x_test, a0_test, parameters_test)
print("a_out[4] = \n", a_out[4])
print("y_pred_out[1] = \n", y_pred_out[1])


In [ ]:
def rnn_cell_backward(da_next, cache):
    """
    단일 RNN 셀의 역전파 단계를 구현합니다.
    """
    (a_next, a_prev, xt, parameters) = cache
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    
    dtanh = (1 - a_next ** 2) * da_next
    dWax = dtanh @ xt.T
    dWaa = dtanh @ a_prev.T
    dba = np.sum(dtanh, axis=1, keepdims=True)
    
    da_prev = Waa.T @ dtanh
    
    return da_prev, dWax, dWaa, dba


In [ ]:
def rnn_backward(da, caches):
    """
    기본 RNN의 역전파 단계를 구현합니다.
    
    매개변수:
    da -- 은닉 상태에 대한 손실의 기울기, 크기 (n_a, m, T_x)인 넘파이 배열
    caches -- 순전파에서 저장된 값들의 튜플, (캐시 리스트, x)를 포함
    
    반환값:
    gradients -- 가중치의 기울기를 포함하는 딕셔너리
    """
    (caches, x) = caches
    n_a, m, T_x = da.shape
    n_x, _, _ = x.shape


In [ ]:
# 기울기 초기화
    gradients = {
        "dWax": np.zeros((n_a, n_x)),
        "dWaa": np.zeros((n_a, n_a)),
        "dba": np.zeros((n_a, 1)),
        "da_prev": np.zeros((n_a, m))
    }
    
    da_next = np.zeros((n_a, m))
    
    # 시간 축을 따라 역전파 수행
    for t in reversed(range(T_x)):
        cache = caches[t]
        da_next, dWax_t, dWaa_t, dba_t = rnn_cell_backward(da[:, :, t] + da_next, cache)
        
        # 기울기 누적
        gradients["dWax"] += dWax_t
        gradients["dWaa"] += dWaa_t
        gradients["dba"] += dba_t
    
    return gradients


In [ ]:
# 역전파 테스트 코드
np.random.seed(2)
da_test = np.random.randn(n_a, m, T_x)
gradients_out = rnn_backward(da_test, caches_out)
print("dWax = \n", gradients_out["dWax"])
print("dWaa = \n", gradients_out["dWaa"])
print("dba = \n", gradients_out["dba"])
